In [1]:
import os
import numpy as np
import librosa


In [2]:
def extract_mfcc_features(file_path, n_mfcc=13):
    y,sr = librosa.load(file_path,sr=16000)
    mfcc = librosa.feature.mfcc(
        y=y,
        sr=sr,
        n_mfcc = n_mfcc,
        n_fft = 2048,
        hop_length = 512
    )

    mfcc_mean = np.mean(mfcc , axis=1)
    mfcc_std = np.std(mfcc, axis =1)

    features = np.hstack((mfcc_mean, mfcc_std))
    return features,mfcc

 

In [3]:
BASE_PATH = "archive/KAGGLE/AUDIO"
fake_file = os.path.join(BASE_PATH,"FAKE",os.listdir(os.path.join(BASE_PATH,"FAKE"))[0])
real_file = os.path.join(BASE_PATH,"REAL",os.listdir(os.path.join(BASE_PATH,"REAL"))[0])

print("Fake file:",fake_file)
print("Real file:",real_file)



Fake file: archive/KAGGLE/AUDIO\FAKE\biden-to-linus.wav
Real file: archive/KAGGLE/AUDIO\REAL\biden-original.wav


In [6]:
fake_features, fake_mfcc = extract_mfcc_features(fake_file)
real_features, real_mfcc = extract_mfcc_features(real_file)

print("Fake MFCC matrix shape:", fake_mfcc.shape)
print("Real MFCC matrix shape:", real_mfcc.shape)

print("\nFake feature vector shape:", fake_features.shape)
print("Real feature vector shape:", real_features.shape)


c:\Users\Lenovo\miniconda3\envs\marine_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Fake MFCC matrix shape: (13, 18751)
Real MFCC matrix shape: (13, 18751)

Fake feature vector shape: (26,)
Real feature vector shape: (26,)


In [5]:
print("Fake MFCC features:\n", fake_features)
print("\nReal MFCC features:\n", real_features)


Fake MFCC features:
 [-234.22937      84.56447     -26.98922      -3.005591    -19.681854
  -13.69309      -9.833106    -14.317146     -0.56719613  -13.9683075
  -10.108655     -5.702584    -10.291721    115.89279      44.943348
   42.760063     30.068499     24.084576     15.587006     14.348217
   18.160917     12.553204     16.953772      9.861683      9.403383
    9.499697  ]

Real MFCC features:
 [-246.27689     94.38078    -53.998814    -6.5312223  -15.407293
   -8.885221   -17.763071    -8.733079    -9.4301405   -2.5279253
   -9.024991    -2.8638422  -13.437067    80.60209     35.175934
   29.298021    21.347929    18.687181    13.966587    14.641085
   12.521326    12.331428     8.847682     9.190603     9.276512
    8.497006 ]


In [8]:
X=[]
y=[]

FAKE_DIR = os.path.join(BASE_PATH,"FAKE")
REAL_DIR = os.path.join(BASE_PATH,"REAL")

#prcoess fake file
for file in os.listdir(FAKE_DIR):
    if file.endswith(".wav"):
        path = os.path.join(FAKE_DIR, file)
        features, _ = extract_mfcc_features(path)
        X.append(features)
        y.append(0)

#process real file

for file in os.listdir(REAL_DIR):
    if file.endswith(".wav"):
        path = os.path.join(REAL_DIR,file)
        features,_ = extract_mfcc_features(path)
        X.append(features)
        y.append(1)


    

In [6]:
print("FAKE files:", os.listdir(FAKE_DIR))
print("REAL files:", os.listdir(REAL_DIR))


FAKE files: ['biden-to-linus.wav', 'biden-to-margot.wav', 'biden-to-musk.wav', 'biden-to-Obama.wav', 'biden-to-ryan.wav', 'biden-to-taylor.wav', 'biden-to-Trump.wav', 'linus-to-biden.wav', 'linus-to-margot.wav', 'linus-to-musk.wav', 'linus-to-obama.wav', 'linus-to-ryan.wav', 'linus-to-taylor.wav', 'linus-to-trump.wav', 'margot-to-biden.wav', 'margot-to-linus.wav', 'margot-to-musk.wav', 'margot-to-obama.wav', 'margot-to-ryan.wav', 'margot-to-taylor.wav', 'margot-to-trump.wav', 'musk-to-biden.wav', 'musk-to-linus.wav', 'musk-to-margot.wav', 'musk-to-obama.wav', 'musk-to-ryan.wav', 'musk-to-taylor.wav', 'musk-to-trump.wav', 'Obama-to-Biden.wav', 'obama-to-linus.wav', 'obama-to-margot.wav', 'obama-to-musk.wav', 'obama-to-ryan.wav', 'obama-to-taylor.wav', 'Obama-to-Trump.wav', 'ryan-to-biden.wav', 'ryan-to-linus.wav', 'ryan-to-margot.wav', 'ryan-to-musk.wav', 'ryan-to-obama.wav', 'ryan-to-taylor.wav', 'ryan-to-trump.wav', 'taylor-to-biden.wav', 'taylor-to-linus.wav', 'taylor-to-margot.wav',

In [9]:
X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:",y.shape)
print("Fake samples:", np.sum(y == 0))
print("Real samples:", np.sum(y == 1))




X shape: (64, 26)
y shape: (64,)
Fake samples: 56
Real samples: 8


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report





TRAIN TEST SPLIT (STRATIFIED)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify = y
)

print("Train samples:", X_train.shape[0])
print("Test samples:",X_test.shape[0])
print("Train FAKE:",np.sum(y_train==0),"REAL:",np.sum(y_train==1))
print("Test Fake:",np.sum(y_test==0),"REAL:",np.sum(y_test==1))

Train samples: 51
Test samples: 13
Train FAKE: 45 REAL: 6
Test Fake: 11 REAL: 2


In [13]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [14]:
svm = SVC(
    kernel="rbf",
    C=1.0,
    gamma="scale",
    class_weight="balanced",
    random_state=42
)

svm.fit(X_train_scaled,y_train)



,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive. The penaltyis a squared l2 penalty. For an intuitive visualization of the effectsof scaling the regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",1.0
,"kernel kernel: {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'} or callable, default='rbf'Specifies the kernel type to be used in the algorithm. Ifnone is given, 'rbf' will be used. If a callable is given it is used topre-compute the kernel matrix from data matrices; that matrix should bean array of shape ``(n_samples, n_samples)``. For an intuitivevisualization of different kernel types see:ref:`sphx_glr_auto_examples_svm_plot_svm_kernels.py`.",'rbf'
,"degree degree: int, default=3Degree of the polynomial kernel function ('poly').Must be non-negative. Ignored by all other kernels.",3
,"gamma gamma: {'scale', 'auto'} or float, default='scale'Kernel coefficient for 'rbf', 'poly' and 'sigmoid'.- if ``gamma='scale'`` (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,- if 'auto', uses 1 / n_features- if float, must be non-negative... versionchanged:: 0.22 The default value of ``gamma`` changed from 'auto' to 'scale'.",'scale'
,"coef0 coef0: float, default=0.0Independent term in kernel function.It is only significant in 'poly' and 'sigmoid'.",0.0
,"shrinking shrinking: bool, default=TrueWhether to use the shrinking heuristic.See the :ref:`User Guide `.",True
,"probability probability: bool, default=FalseWhether to enable probability estimates. This must be enabled priorto calling `fit`, will slow down that method as it internally uses5-fold cross-validation, and `predict_proba` may be inconsistent with`predict`. Read more in the :ref:`User Guide `.",False
,"tol tol: float, default=1e-3Tolerance for stopping criterion.",0.001
,"cache_size cache_size: float, default=200Specify the size of the kernel cache (in MB).",200
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to class_weight[i]*C forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",'balanced'
,"verbose verbose: bool, default=FalseEnable verbose output. Note that this setting takes advantage of aper-process runtime setting in libsvm that, if enabled, may not workproperly in a multithreaded context.",False


In [15]:
y_pred = svm.predict(X_test_scaled)
acc = accuracy_score(y_test,y_pred)
print("Test Accuracy:", acc)

print("\n Classification Report:")
print(classification_report(y_test,y_pred,target_names=["FAKE","REAL"]))

print("\n Confusion Matrix:")
print(confusion_matrix(y_test,y_pred))

Test Accuracy: 0.8461538461538461

 Classification Report:
              precision    recall  f1-score   support

        FAKE       0.85      1.00      0.92        11
        REAL       0.00      0.00      0.00         2

    accuracy                           0.85        13
   macro avg       0.42      0.50      0.46        13
weighted avg       0.72      0.85      0.78        13


 Confusion Matrix:
[[11  0]
 [ 2  0]]


c:\Users\Lenovo\miniconda3\envs\marine_env\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Lenovo\miniconda3\envs\marine_env\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Lenovo\miniconda3\envs\marine_env\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize